In [1]:
import os
import sys
import configparser

configread=configparser.ConfigParser()
configread.read(r'../config.ini')
s_path=configread.get('spark_env','spark_env_file')
h_path=configread.get('hadoop_pth','hadoop_pth_file')
sp_bin=configread.get('spark_bin','spark_bin_file')
sp_py=configread.get('spark_py','spark_py_file')
sp_pysp=configread.get('spark_pysp','spark_pysp_file')
sp_pylib=configread.get('spark_pylib','spark_pylib_file')
sp_pylib_pysp=configread.get('spark_pylib_pysp','spark_pylib_pysp_file')
sp_pylib_py4j=configread.get('spark_pylib_py4j','spark_pylib_py4j_file')
h_bin=configread.get('hadoop_bin','hadoop_bin_file')
charac=configread.get('input_character','character')
play=configread.get('intput_player','player')

spark_path = s_path
hadoop_path = h_path
os.environ['SPARK_HOME'] = spark_path
os.environ['HADOOP_HOME'] = hadoop_path
sys.path.append(sp_bin)
sys.path.append(sp_py)
sys.path.append(sp_pysp)
sys.path.append(sp_pylib)
sys.path.append(sp_pylib_pysp)
sys.path.append(sp_pylib_py4j)
sys.path.append(h_bin)

In [50]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

In [51]:
conf=SparkConf().setAppName("sql_number_functions_using_RDDs").setMaster('local[4]')
spark=SparkSession.builder.appName('sql_number_functions_using_RDDs').master('local[4]').config("spark.some.config.option", "some-value").getOrCreate()

In [53]:
sc=spark.sparkContext

In [54]:
c=sc.textFile(charac,3)
p=sc.textFile(play,3)

In [55]:
c.collect()

['id,player_id,name,level,class,account_balance,hp,mp,strength,wisdom,stat_modifier,weight,height',
 '1,1,Kav,3,wizard,899.34,100,200,12,20,1.05,65,1.72',
 '2,2,Gniok,5,warrior,423.83,170,80,20,23,4.57,101.4,2',
 '3,1,Mnah,2,warrior,0.00,140,60,16,20,-1.00,80.300003,1.9299999',
 '4,2,Balryll,1,warrior,723.12,130,50,15,0,2.89,78.900002,1.87',
 '5,3,Mnadjaid,13,wizard,4321.93,171,37,86,-4,61.0358,1.67,null',
 '6,4,Duroso,24,wizard,7231.32,240,890,45,104,17.18,0.2,1',
 '7,5,Rolamar,3,warrior,753.42,150,65,18,21,-29.99,91.300003,1.85',
 '8,5,Avealath,7,warrior,1023.00,190,95,28,29,17.44,95.199997,1.9']

In [56]:
p.collect()

['id,firstname,lastname',
 '1,Alan,Gilman',
 '2,Kate,Friss',
 '3,Anne,Birch',
 '4,Adrian,Regos',
 '5,Stephen,Colvin']

In [57]:
c1=c.map(lambda x:x.split(','))
p1=p.map(lambda x:x.split(','))
h1=c1.first()
h2=p1.first()
c1=c1.filter(lambda x:x!=h1)
p1=p1.filter(lambda x:x!=h2)

In [58]:
# For each character, show its name, its account_balance and its account_balance rounded to the nearest integer (name this column rounded_account_balance). 
#Notice how mathematical rounding is applied.

# USING RDD

In [86]:
c1.map(lambda x:(x[2],float(x[5]),round(float(x[5])))).collect()

[('Kav', 899.34, 899),
 ('Gniok', 423.83, 424),
 ('Mnah', 0.0, 0),
 ('Balryll', 723.12, 723),
 ('Mnadjaid', 4321.93, 4322),
 ('Duroso', 7231.32, 7231),
 ('Rolamar', 753.42, 753),
 ('Avealath', 1023.0, 1023)]

In [87]:
# round with precision 1

c1.map(lambda x:(x[2],float(x[5]),round(float(x[5]),1))).collect()

[('Kav', 899.34, 899.3),
 ('Gniok', 423.83, 423.8),
 ('Mnah', 0.0, 0.0),
 ('Balryll', 723.12, 723.1),
 ('Mnadjaid', 4321.93, 4321.9),
 ('Duroso', 7231.32, 7231.3),
 ('Rolamar', 753.42, 753.4),
 ('Avealath', 1023.0, 1023.0)]

In [60]:
# USING DATAFRAME

In [77]:
cdf=spark.read.format('csv').option('delimiter',',').option('header',True).option('inferschema',True).load(charac)
pdf=spark.read.format('csv').option('delimiter',',').option('header',True).option('inferschema',True).load(play)

In [78]:
cdf.show(truncate=False)
pdf.show(truncate=False)

+---+---------+--------+-----+-------+---------------+---+---+--------+------+-------------+---------+---------+
|id |player_id|name    |level|class  |account_balance|hp |mp |strength|wisdom|stat_modifier|weight   |height   |
+---+---------+--------+-----+-------+---------------+---+---+--------+------+-------------+---------+---------+
|1  |1        |Kav     |3    |wizard |899.34         |100|200|12      |20    |1.05         |65.0     |1.72     |
|2  |2        |Gniok   |5    |warrior|423.83         |170|80 |20      |23    |4.57         |101.4    |2        |
|3  |1        |Mnah    |2    |warrior|0.0            |140|60 |16      |20    |-1.0         |80.300003|1.9299999|
|4  |2        |Balryll |1    |warrior|723.12         |130|50 |15      |0     |2.89         |78.900002|1.87     |
|5  |3        |Mnadjaid|13   |wizard |4321.93        |171|37 |86      |-4    |61.0358      |1.67     |null     |
|6  |4        |Duroso  |24   |wizard |7231.32        |240|890|45      |104   |17.18        |0.2 

In [83]:
import pyspark.sql.functions as f
cdf.select(cdf['name'],cdf['account_balance'],f.round(cdf['account_balance']).alias('rounded_account_balance')).show()

+--------+---------------+-----------------------+
|    name|account_balance|rounded_account_balance|
+--------+---------------+-----------------------+
|     Kav|         899.34|                  899.0|
|   Gniok|         423.83|                  424.0|
|    Mnah|            0.0|                    0.0|
| Balryll|         723.12|                  723.0|
|Mnadjaid|        4321.93|                 4322.0|
|  Duroso|        7231.32|                 7231.0|
| Rolamar|         753.42|                  753.0|
|Avealath|         1023.0|                 1023.0|
+--------+---------------+-----------------------+



In [89]:
# round with precision 1

cdf.select(cdf['name'],cdf['account_balance'],f.round(cdf['account_balance'],1).alias('rounded_account_balance')).show()

+--------+---------------+-----------------------+
|    name|account_balance|rounded_account_balance|
+--------+---------------+-----------------------+
|     Kav|         899.34|                  899.3|
|   Gniok|         423.83|                  423.8|
|    Mnah|            0.0|                    0.0|
| Balryll|         723.12|                  723.1|
|Mnadjaid|        4321.93|                 4321.9|
|  Duroso|        7231.32|                 7231.3|
| Rolamar|         753.42|                  753.4|
|Avealath|         1023.0|                 1023.0|
+--------+---------------+-----------------------+

